In [ ]:
# import os
# import sys

# sys.dont_write_bytecode = True
# os.environ["NUMBA_DISABLE_JIT"] = "1"

In [ ]:
import pandas as pd
import numpy as np
from quantfreedom.class_practice.enums import *
from quantfreedom.class_practice.base import backtest_df_only, backtest_sim_6, plot_one_result
from quantfreedom.class_practice.helper_funcs import create_os_cart_product_nb
from quantfreedom.indicators.talib_ind import from_talib



price_data = pd.read_hdf("../../tests/data/prices.hd5")
%load_ext autoreload
%autoreload 2


In [ ]:
price_data['BTCUSDT']

In [ ]:
rsi_ind = from_talib(
    func_name='rsi',
    nickname='rsi',
    price_data=price_data,
    input_names = ['close'],
    parameters = {'timeperiod': [15, 20]}
)
entries = rsi_ind.is_below(
    user_args=[40, 50],
    candle_ohlc="close",
)

In [ ]:
account_state = AccountState()
backtest_settings = BacktestSettings(
    divide_records_array_size_by=2,
    gains_pct_filter=10.0,
    total_trade_filter=10,
    upside_filter=0.0,
)
exchange_settings = ExchangeSettings()
order_settings_arrays = OrderSettingsArrays(
    risk_account_pct_size=np.array([1.0]) / 100,
    sl_based_on_add_pct=np.array([0.01, .02]) / 100,
    sl_based_on_lookback=np.array([50]),
    risk_reward=np.array([2.0, 5.0]),
    leverage_type=np.array([LeverageType.Dynamic]),
    sl_candle_body_type=np.array([CandleBodyType.Low]),
    increase_position_type=np.array([IncreasePositionType.RiskPctAccountEntrySize]),
    stop_loss_type=np.array([StopLossType.SLBasedOnCandleBody]),
    take_profit_type=np.array([TakeProfitType.RiskReward]),
    max_equity_risk_pct=np.array([3]) / 100,
    order_type=np.array([OrderType.Long]),
    sl_to_be_based_on_candle_body_type=np.array([CandleBodyType.Nothing]),
    sl_to_be_when_pct_from_candle_body=np.array([0.0]) / 100,
    sl_to_be_zero_or_entry_type=np.array([SLToBeZeroOrEntryType.Nothing]),
    trail_sl_based_on_candle_body_type=np.array([CandleBodyType.High]),
    trail_sl_when_pct_from_candle_body=np.array([3.0]) / 100,
    trail_sl_by_pct=np.array([1.0]) / 100,
    static_leverage=np.array([0.0]),
    tp_fee_type=np.array([TakeProfitFeeType.Limit]),
)

In [ ]:
os_cart_arrays = create_os_cart_product_nb(order_settings_arrays=order_settings_arrays)

In [ ]:
list(price_data.columns.levels[0])

In [ ]:
strat_res_df = backtest_df_only(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    backtest_settings=backtest_settings,
    exchange_settings=exchange_settings,
    price_data=price_data['EOSUSD'],
    entries=entries['EOSUSD'],
)

In [ ]:
strat_res_df

In [ ]:
order_records_df = backtest_sim_6(
    account_state=account_state,
    os_cart_arrays=os_cart_arrays,
    exchange_settings=exchange_settings,
    price_data=price_data["EOSUSD"],
    entries=entries["EOSUSD"],
    strat_indexes=[0, 2, 4],
    strat_res_df=strat_res_df,
)

In [ ]:
order_records_df

In [ ]:
plot_one_result(
    strat_res_df=strat_res_df,
    strat_result_index=0,
    price_data=price_data["EOSUSD"],
    order_records_df=order_records_df,
)